In [ ]:
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

In [ ]:
# Constants
INPUT_DATA_PATH = "/kaggle/input/ashrae-energy-prediction"

MIN_TRAIN_TIMESTAMP = pd.Timestamp("2016-01-01 00:00:00")
MAX_TRAIN_TIMESTAMP = pd.Timestamp("2016-12-31 23:00:00")
TRAIN_DATA_RESOLUTION = "1h"

## Load raw data

In [ ]:
# Meter readings
readings_df = pd.read_csv(
    f"{INPUT_DATA_PATH}/train.csv",
    header=0,
    names=["building_id", "meter_id", "timestamp", "meter_reading"],
)
readings_df["timestamp"] = pd.to_datetime(readings_df["timestamp"])

# Buildings
buildings_df = pd.read_csv(f"{INPUT_DATA_PATH}/building_metadata.csv")

# Weather
weather_df = pd.read_csv(f"{INPUT_DATA_PATH}/weather_train.csv")
weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])

## Preprocess weather data

In [ ]:
WEATHER_FEATURE_COLUMNS = [
    'air_temperature',
    'cloud_coverage',
    'dew_temperature',
    'precip_depth_1_hr',
    'sea_level_pressure',
    'wind_direction',
    'wind_speed'
]

In [ ]:
def reindex_weather_data(weather_df: pd.DataFrame) -> pd.DataFrame:
    # Reindex weather data such that every site has a measurement
    # for each training timestamp
    weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])
    timestamps = pd.date_range(
        MIN_TRAIN_TIMESTAMP,
        MAX_TRAIN_TIMESTAMP,
        freq=TRAIN_DATA_RESOLUTION,
        inclusive="both"
    )
    timestamps = pd.DatetimeIndex(timestamps, name="timestamp")
    site_dfs = []
    for site_id, site_df in weather_df.groupby("site_id", observed=True):
        site_df = site_df.set_index("timestamp").reindex(timestamps).reset_index()
        site_df["site_id"] = site_df["site_id"].fillna(value=site_id)
        site_dfs.append(site_df)

    weather_df = pd.concat(site_dfs, ignore_index=True)
    return weather_df


def interpolate(weather_df: pd.DataFrame, column: str) -> pd.DataFrame:
    weather_df[column] = weather_df[column].interpolate("linear", limit=12)
    weather_df[column] = weather_df[column].ffill(limit=2)
    weather_df[column] = weather_df[column].bfill(limit=2)
    return weather_df


def _mean_weather_by_date_and_site(weather_df, column) -> pd.DataFrame:
    mean_values = (
        weather_df
        .groupby(["date", "site_id"])[[column]]
        .mean()
        .reset_index()
    )
    return mean_values


def _merge_onto_weather_df(weather_df, right, right_suffix) -> pd.DataFrame:
    weather_df = weather_df.merge(
        right=right,
        how="left",
        on=["date", "site_id"],
        suffixes=("", right_suffix)
    )
    return weather_df


def impute_with_same_day_mean(weather_df: pd.DataFrame, column: str) -> pd.DataFrame:
    # Compute same day mean values and merge onto weather df
    mean_values = _mean_weather_by_date_and_site(weather_df, column)
    weather_df = _merge_onto_weather_df(weather_df, mean_values, "_mean")
    
    # Fill with means from same day
    weather_df[column] = weather_df[column].fillna(weather_df[f"{column}_mean"])
    weather_df = weather_df.drop(f"{column}_mean", axis=1)
    return weather_df


def ffill_mean_by_date(weather_df: pd.DataFrame, column: str) -> pd.DataFrame:
    # Compute same day mean values and merge onto weather df
    mean_values = _mean_weather_by_date_and_site(weather_df, column)
    
    # ffill by site and date
    site_dfs = []
    for site_id, site_df in mean_values.groupby("site_id"):
        site_df = site_df.sort_values("date")
        site_df[column] = site_df[column].ffill().bfill()
        site_dfs.append(site_df)
    mean_values = pd.concat(site_dfs, ignore_index=True)
    
    # Merge back onto main and fill with mean values
    weather_df = _merge_onto_weather_df(weather_df, mean_values, "_mean")
    weather_df[column] = weather_df[column].fillna(weather_df[f"{column}_mean"])
    weather_df = weather_df.drop(f"{column}_mean", axis=1)
    return weather_df


def fill_missing_weather_data(weather_df: pd.DataFrame) -> pd.DataFrame:
    weather_df["date"] = weather_df["timestamp"].dt.date

    for column in WEATHER_FEATURE_COLUMNS:
        weather_df = interpolate(weather_df, column)
        weather_df = impute_with_same_day_mean(weather_df, column)
        weather_df = ffill_mean_by_date(weather_df, column)

    weather_df = weather_df.drop(columns=["date"])
    
    return weather_df

In [ ]:
weather_df = reindex_weather_data(weather_df)
weather_df = fill_missing_weather_data(weather_df)

## Weather feature engineering

In [ ]:
def add_smoothed_weather_feature(df: pd.DataFrame, feature: str) -> pd.DataFrame:
    site_dfs = []
    for site_id, site_df in df.groupby("site_id"):
        site_df = site_df[["site_id", "timestamp", feature]]
        site_df = site_df.drop_duplicates(keep="first")
        site_df[f"{feature}_smoothed"] = savgol_filter(
            np.array(site_df[feature]),
            window_length=12,
            polyorder=2,
        )
        site_dfs.append(site_df)
    site_dfs = pd.concat(site_dfs, ignore_index=True).drop(columns=[feature])
    df = df.merge(right=site_dfs, on=["site_id", "timestamp"], how="left")
    return df


def add_lagged_weather_feature(df: pd.DataFrame, feature: str, lag: int) -> pd.DataFrame:
    site_dfs = []
    for site_id, site_df in df.groupby("site_id"):
        site_df = site_df[["site_id", "timestamp", feature]]
        site_df = site_df.drop_duplicates(keep="first")
        site_df[f"{feature}_lag_{lag}"] = site_df[feature].shift(lag)
        site_dfs.append(site_df)
    site_dfs = pd.concat(site_dfs, ignore_index=True).drop(columns=[feature])
    df = df.merge(right=site_dfs, on=["site_id", "timestamp"], how="left")
    return df
    

def add_rolling_mean_weather_feature(df: pd.DataFrame, feature_name: str) -> pd.DataFrame:
    # Do this by site!!
    ...

In [ ]:
# Air temperature

weather_df = add_smoothed_weather_feature(weather_df, "air_temperature")
weather_df = add_lagged_weather_feature(weather_df, "air_temperature", 1)
weather_df = add_lagged_weather_feature(weather_df, "air_temperature", 2)


# Dew temperature
...


# Sea level pressure
...

In [ ]:
weather_df

## Merge

In [ ]:
def merge_dfs(readings_df: pd.DataFrame, buildings_df: pd.DataFrame, weather_df: pd.DataFrame) -> pd.DataFrame:
    # Cast merge cols to the same type
    readings_df["building_id"] = readings_df["building_id"].astype(int)
    readings_df["timestamp"] = pd.to_datetime(readings_df["timestamp"])

    buildings_df["building_id"] = buildings_df["building_id"].astype(int)
    buildings_df["site_id"] = buildings_df["site_id"].astype(int)

    weather_df["site_id"] = weather_df["site_id"].astype(int)
    weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])

    # Merge
    merged_df = pd.merge(left=readings_df, right=buildings_df, how="left", on="building_id")
    merged_df = pd.merge(left=merged_df, right=weather_df, how="left", on=["site_id", "timestamp"])

    return merged_df

In [ ]:
train_df = merge_dfs(readings_df, buildings_df, weather_df)

## Feature engineering

In [ ]:
def add_periodic_features(df: pd.DataFrame, feature: str, period: int) -> pd.DataFrame:
    df[f"{feature}_sin"] = np.sin(2 * np.pi * df[feature] / period)
    df[f"{feature}_cos"] = np.cos(2 * np.pi * df[feature] / period)
    return df


In [ ]:
# Meter reading features

def kbtu_to_kwh(df: pd.DataFrame) -> pd.DataFrame:
    mask = (df["building_id"] == 0) & (df["meter_id"] == 0)
    df.loc[mask, "meter_reading"] = df.loc[mask, "meter_reading"] * 0.2931
    return df


def add_temporal_features(df: pd.DataFrame) -> pd.DataFrame:
    df["hour"] = df["timestamp"].dt.hour
    df = add_periodic_features(df, "hour", 24)
    
    df["day_of_week"] = df["timestamp"].dt.weekday
    df = add_periodic_features(df, "day_of_week", 7)
    
    df["month"] = df["timestamp"].dt.month
    df = add_periodic_features(df, "month", 12)
    
    df["is_weekend"] = (df["timestamp"].dt.weekday >= 5).astype(float)
    
    return df

In [ ]:
train_df = kbtu_to_kwh(train_df)
train_df = add_temporal_features(train_df)

In [ ]:
# Building features

def add_building_age_feature(df: pd.DataFrame) -> pd.DataFrame:
    df["building_age_years"] = df["timestamp"].dt.year - df["year_built"]
    return df


def add_building_area_feature(df: pd.DataFrame) -> pd.DataFrame:
    df["building_area_square_feet"] = df["square_feet"] * df["floor_count"]
    return df

In [ ]:
train_df = add_building_age_feature(train_df)
train_df = add_building_area_feature(train_df)

In [ ]:
# Weather features

# https://www.kaggle.com/code/selfishgene/filtering-and-auto-correlation-tutorial

WEATHER_FEATURE_COLUMNS = [
    'air_temperature',  # smooth out using sg filter + rolling 24hr mean
    'cloud_coverage',
    'dew_temperature',  # smooth out using sg filter
    'precip_depth_1_hr',
    'sea_level_pressure',  # smooth out using sg filter + rolling 24hr mean
    'wind_direction',  # sin / cos 
    'wind_speed'
]


def feels_like_temperature_feature(df: pd.DataFrame) -> pd.DataFrame:
    ...


def wet_bulb_temperature_feature(df: pd.DataFrame) -> pd.DataFrame:
    ...


def cooling_degree_days(df: pd.DataFrame) -> pd.DataFrame:
    # https://www.investopedia.com/terms/c/colddegreeday.asp
    ...


def heating_degree_days():
    ...

In [ ]:
def target_transform(df: pd.DataFrame) -> pd.DataFrame:
    df["log_meter_reading"] = np.log1p(np.array(df["meter_reading"]))
    return df